##### Copyright 2020 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# [TensorBoard.dev](https://tensorboard.dev) を使う

[TensorBoard.dev](https://tensorboard.dev) は、無料で提供されている一般向けの [TensorBoard](https://tensorflow.org/tensorboard) サービスです。機械学習の実験をアップロードし、あらゆるユーザーと共有することができます。

このノートブックでは、簡単なモデルをトレーニングし、TensorBoard.dev にログをアップロードする方法を学習します。[プレビュー](https://tensorboard.dev/experiment/rldGbR8rRHeCEbkK61SWTQ)

### セットアップとインポート

このノートブックでは、バージョン `2.3.0` 以降でのみ利用できる TensorBoard 機能を使用します。

In [ ]:
import tensorflow as tf
import datetime
from tensorboard.plugins.hparams import api as hp

### 簡単なモデルのトレーニングと TensorBoard ログの作成

In [ ]:
mnist = tf.keras.datasets.mnist

(x_train, y_train),(x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

def create_model():
  return tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape=(28, 28)),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(10, activation='softmax')
  ])

TensorBoard ログは、トレーニング中に [TensorBoard](https://www.tensorflow.org/tensorboard/get_started) と [ハイパーパラメータコールバック](https://www.tensorflow.org/tensorboard/hyperparameter_tuning_with_hparams) を Keras の Model.fit() に渡して作成します。作成後は、そのログを TensorBoard.dev にアップロードすることができます。


In [ ]:
model = create_model()
model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy'])

log_dir="logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(
    log_dir=log_dir, histogram_freq=1)
hparams_callback = hp.KerasCallback(log_dir, {
    'num_relu_units': 512,
    'dropout': 0.2
})

model.fit(
    x=x_train, 
    y=y_train, 
    epochs=5, 
    validation_data=(x_test, y_test), 
    callbacks=[tensorboard_callback, hparams_callback])

### （Jupyter 限定）TensorBoard.dev の認証

**Colab では、このステップは不要です。**

このステップには、Jupyter の外部でシェルコンソールを使って認証する必要があります。ご利用のコンソールで、次のコマンドを実行してください。

`tensorboard dev list`

このフローの一環として、認証コードが提供されます。このコードは、サービス規約に同意する際に必要となります。

### TensorBoard.dev へのアップロード

TensorBoard ログをアップロードすると、ほかの人に共有できる URL が提示されます。

アップロードした TensorBoards は一般に公開されるため、機密データはアップロードしないようにしてください。

logdir 全体のアップロードが完了すると、アップローダは終了します。（この動作は、`--one_shot` フラグによって指定されています。）

In [ ]:
!tensorboard dev upload --logdir ./logs \
  --name "Simple experiment with MNIST" \
  --description "Training results from https://colab.sandbox.google.com/github/tensorflow/tensorboard/blob/master/docs/tbdev_getting_started.ipynb" \
  --one_shot

各アップロードには、一意の実験 ID があり、同じディレクトリで新しいアップロードを開始する場合には、新しい実験 ID が与えられます。アップロードしたすべての実験は、https://tensorboard.dev/experiments/ で表示できます。または、次のコマンドを使用して、ターミナルに実験を一覧表示することもできます。

```
tensorboard dev list
```

In [ ]:
!tensorboard dev list

### TensorBoard.dev のスクリーンショット

https://tensorboard.dev/experiments/: に移動すると、次のように表示されます。

![screenshot of TensorBoard.dev](https://gitlocalize.com/repo/4592/ja/site/en-snapshot/tensorboard/images/tbdev_getting_started.png "TensorBoard.dev screenshot")

TensorBoard.dev で新しい実験に移動すると、次のように表示されます。

![screenshot of TensorBoard.dev experiment dashboard](https://github.com/tensorflow/docs-l10n/blob/master/site/ja/tensorboard/images/tbdev_getting_started.png?raw=true)

### TensorBoard.dev 実験の削除

アップロード済みの実験を削除するには、`delete` コマンドを使用して、削除する `experiment_id` を指定します。上記のスクリーンショットでは、experiment_id は左下（`w1lkBAOrR4eH35Y7Lg1DQQ`）に示されています。

In [ ]:
# You must replace YOUR_EXPERIMENT_ID with the value output from the previous
# tensorboard `list` command or `upload` command.  For example
# `tensorboard dev delete --experiment_id pQpJNh00RG2Lf1zOe9BrQA`

## !tensorboard dev delete --experiment_id YOUR_EXPERIMENT_ID_HERE